In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import numpy as np
import scipy
import mne
import matplotlib.pyplot as plt
import os
import pickle

In [2]:
def eeg_data_extraction(data):
    max_time_len = 20000
    raw_data = data.get_data()
    coi = ['Fp1','Fp2','F7','F8','Fz','C3','C4','F3','F4']

    coi_indexes = []
    for channel_name in coi:
        index = data.ch_names.index(channel_name)
        coi_indexes.append(index)
    
    ft_time_series = []
    for channel in coi_indexes:
        time_serie = raw_data[channel][:max_time_len]
        fourier_transform = scipy.fft(time_serie)
        ft_time_series.append(fourier_transform)
    
    frequencies = []
    freq_spectra = []
    for channel in range(len(ft_time_series)):
        x_axis = np.arange(0,len(ft_time_series[channel]))
        N = ft_time_series[channel].shape[0]
        T = 1.0 / 1000.0
        xf = scipy.fftpack.fftfreq(N, T)[:N//2]
        lower_bound = xf>4
        upper_bound = xf<100
        bounds = lower_bound & upper_bound
        non_zero_indices = np.where(bounds,ft_time_series[channel][:N//2],0).nonzero()
        frequencies.append(xf[non_zero_indices])
        freq_spectra.append(np.abs(ft_time_series[channel])[non_zero_indices])
    return frequencies,freq_spectra

In [3]:
def check(string, sub_str): 
    if (string.find(sub_str) == -1): 
        return(False) 
    else: 
        return(True)

def list_files_with_string(files, string):
    output_files = []
    for file in files:
        if check(file, string):
            output_files.append(file)
    return output_files

In [4]:
folders = os.listdir()
folders = list_files_with_string(folders, 'sub')
file_paths=[]
for folder in folders:
    sub_path = './' + folder + '/eeg/'
    files = os.listdir(sub_path)
    edf_files = list_files_with_string(files, '.edf')
    for file in edf_files:
        file_paths.append(sub_path + file)

In [7]:
dictionary_classification = {0: 'generated',1: 'classical'}
generated_music_files1 = list_files_with_string(file_paths, 'genMusic01')
generated_music_files2 = list_files_with_string(file_paths, 'genMusic02')
generated_music_files = generated_music_files1
generated_music_files.extend(generated_music_files2)
classical_music_files = list_files_with_string(file_paths, 'classicalMusic')
file_classifications = []
for file in generated_music_files:
    filename = './spec_files/' + file.split('/')[-1][:-4] + '.pck'
    file_classifications.append([filename, 0])
for file in classical_music_files:
    filename = './spec_files/' + file.split('/')[-1][:-4] + '.pck'
    file_classifications.append([filename, 1])

In [8]:
generated_music_files

['./sub-01/eeg/sub-01_task-genMusic01_eeg.edf',
 './sub-02/eeg/sub-02_task-genMusic01_eeg.edf',
 './sub-03/eeg/sub-03_task-genMusic01_eeg.edf',
 './sub-04/eeg/sub-04_task-genMusic01_eeg.edf',
 './sub-05/eeg/sub-05_task-genMusic01_eeg.edf',
 './sub-06/eeg/sub-06_task-genMusic01_eeg.edf',
 './sub-07/eeg/sub-07_task-genMusic01_eeg.edf',
 './sub-08/eeg/sub-08_task-genMusic01_eeg.edf',
 './sub-09/eeg/sub-09_task-genMusic01_eeg.edf',
 './sub-10/eeg/sub-10_task-genMusic01_eeg.edf',
 './sub-11/eeg/sub-11_task-genMusic01_eeg.edf',
 './sub-12/eeg/sub-12_task-genMusic01_eeg.edf',
 './sub-13/eeg/sub-13_task-genMusic01_eeg.edf',
 './sub-14/eeg/sub-14_task-genMusic01_eeg.edf',
 './sub-15/eeg/sub-15_task-genMusic01_eeg.edf',
 './sub-16/eeg/sub-16_task-genMusic01_eeg.edf',
 './sub-17/eeg/sub-17_task-genMusic01_eeg.edf',
 './sub-18/eeg/sub-18_task-genMusic01_eeg.edf',
 './sub-19/eeg/sub-19_task-genMusic01_eeg.edf',
 './sub-20/eeg/sub-20_task-genMusic01_eeg.edf',
 './sub-21/eeg/sub-21_task-genMusic01_ee

In [9]:
import csv

with open("annotations.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(file_classifications)

In [7]:
eeg_data = []
for file in file_paths:
    data = mne.io.read_raw_edf(file)
    filename = './spec_files/' + file.split('/')[-1][:-4] + '.pck'
    xf, freq_spectra = eeg_data_extraction(data)
    with open(filename, 'wb') as handle:
        pickle.dump(freq_spectra, handle, protocol=pickle.HIGHEST_PROTOCOL)

Extracting EDF parameters from C:\Users\usr1\Documents\NMA Project\sub-01\eeg\sub-01_task-classicalMusic_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-7-f7b0623b84e2>:3: RuntimeWarning: Physical range is not defined in following channels:
sams_valence, sams_arousal, sams_valencert, sams_arousalrt, nback_stimuli, nback_keypress
  data = mne.io.read_raw_edf(file)


Extracting EDF parameters from C:\Users\usr1\Documents\NMA Project\sub-01\eeg\sub-01_task-genMusic01_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-2-5665553f7c5d>:14: DeprecationWarning: Using scipy.fft as a function is deprecated and will be removed in SciPy 1.5.0, use scipy.fft.fft instead.
  fourier_transform = scipy.fft(time_serie)
<ipython-input-7-f7b0623b84e2>:3: RuntimeWarning: Physical range is not defined in following channels:
sams_valence, sams_arousal, sams_valencert, sams_arousalrt, nback_stimuli, nback_keypress
  data = mne.io.read_raw_edf(file)


Extracting EDF parameters from C:\Users\usr1\Documents\NMA Project\sub-01\eeg\sub-01_task-genMusic02_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


...

<ipython-input-7-f7b0623b84e2>:3: RuntimeWarning: Physical range is not defined in following channels:
sams_valence, sams_arousal, sams_valencert, sams_arousalrt, nback_stimuli, nback_keypress
  data = mne.io.read_raw_edf(file)


Extracting EDF parameters from C:\Users\usr1\Documents\NMA Project\sub-21\eeg\sub-21_task-genMusic03_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-7-f7b0623b84e2>:3: RuntimeWarning: Physical range is not defined in following channels:
sams_valence, sams_arousal, sams_valencert, sams_arousalrt, nback_stimuli, nback_keypress
  data = mne.io.read_raw_edf(file)


Extracting EDF parameters from C:\Users\usr1\Documents\NMA Project\sub-21\eeg\sub-21_task-washout_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-7-f7b0623b84e2>:3: RuntimeWarning: Physical range is not defined in following channels:
ft_valance, ft_arousal, ft_x, ft_y, ft_ghostvalence, ft_ghostarousal, music, trialtype, sams_valence, sams_arousal, sams_valencert, sams_arousalrt
  data = mne.io.read_raw_edf(file)


In [ ]:
data_dic = {'classification': file_classifications,
            'eeg_spectra': eeg_data,
            'frequencies': xf[0]}

In [ ]:
len(data_dic['eeg_spectra'][0][1][0])

In [ ]:
len(data_dic['frequencies'])

In [ ]:
#saving data
with open('data_for_model.pickle', 'wb') as handle:
    pickle.dump(data_dic, handle, protocol=pickle.HIGHEST_PROTOCOL)

#reading data
with open('data_for_model.pickle', 'rb') as handle:
    test = pickle.load(handle)

In [ ]:
test

In [ ]:
for channel in freq_spectra:
    plt.yscale('log')
    plt.xscale('log')
    plt.plot(xf[0], np.abs(channel))
    plt.show()